In [21]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
load_dotenv()

True

In [2]:
llm=ChatGroq(model='llama-3.1-70b-versatile')

In [3]:
from langchain.schema import Document
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import os
import glob

In [4]:
# model=ChatGroq(model='llama-3.1-70b-versatile')
db_name="vector_db"

In [5]:
# folders = r'C:\Users\varsh\Documents\AI\LLM\RAG_youtube_langchain\*'
folders=glob.glob("knowledge_base/*")

documents=[]
for folder in folders:
    print(folder)
    doc_type=os.path.basename(folder)
    loader=DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader,loader_kwargs={'encoding': 'utf-8'})
    folder_docs=loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"]=doc_type
        documents.append(doc)

knowledge_base\company
knowledge_base\contracts
knowledge_base\employees
knowledge_base\products


In [6]:
# documents

In [7]:
text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks=text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [8]:
len(chunks)

123

In [9]:
doc_types=set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: products, contracts, company, employees


In [10]:
model_name="sentence-transformers/all-mpnet-base-v2"
model_kwargs={"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

C:\Users\varsh\AppData\Local\Temp\ipykernel_14864\656995169.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
c:\Users\varsh\Documents\AI\LLM\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
vectorstore=Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 123 documents


In [12]:
#get one vector and find how many dimensions it has

collection=vectorstore._collection
sample_embedding=collection.get(limit=1, include=['embeddings'])['embeddings'][0]
dimensions=len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 768 dimensions


In [13]:
# sample_embedding
# collection

# Visualizing vector store

In [14]:
result=collection.get(include=['embeddings','documents','metadatas'])
vectors=np.array(result['embeddings'])
doc_types=[metadata['doc_type'] for metadata in result['metadatas']]
colors=[['blue','green','red','orange'][['products','employees','contracts','company'].index(t)] for t in doc_types]

In [15]:
#Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochaistic neighbor embedding)

tsne=TSNE(n_components=2, random_state=42)
reduced_vectors=tsne.fit_transform(vectors)

#Create the 2D scatter plot
fig=go.Figure(data=[go.Scatter(
    x=reduced_vectors[:,0],
    y=reduced_vectors[:,1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {str(d)[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20,b=10,l=10,t=40)
)

fig.show()

In [20]:
tsne=TSNE(n_components=3, random_state=42)
reduced_vectors=tsne.fit_transform(vectors)

#Create the 2D scatter plot
fig=go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:,0],
    y=reduced_vectors[:,1],
    z=reduced_vectors[:,2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {str(d)[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y',zaxis_title='z'),
    width=800,
    height=600,
    margin=dict(r=20,b=10,l=10,t=40)
)

fig.show()

In [22]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever=vectorstore.as_retriever()
conversation_chain=ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\varsh\AppData\Local\Temp\ipykernel_14864\3318285081.py:1: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [24]:
query="Can you describe Insurellm in a few sentences"
result=conversation_chain.invoke({"question": query})
print(result['answer'])

Insurellm is an innovative insurance tech firm with 200 employees across the US, offering four insurance software products and serving over 300 clients worldwide.


In [25]:
def chat(message, history):
    result=conversation_chain.invoke({"question": message})
    return result['answer']

In [27]:
import gradio as gr
view=gr.ChatInterface(chat).launch()

c:\Users\varsh\Documents\AI\LLM\.venv\Lib\site-packages\gradio\components\chatbot.py:228: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [32]:
# Let's investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever=vectorstore.as_retriever()
conversation_chain=ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory,
                                                        callbacks=[StdOutCallbackHandler()])
query="Who received the prestigious IIOTY award in 2023?"
result=conversation_chain.invoke({"question": query})
answer=result['answer']
print('\nAnswer:', answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Annual Performance History
- **2020:**  
  - Completed onboarding successfully.  
  - Met expectations in delivering project milestones.  
  - Received positive feedback from the team leads.

- **2021:**  
  - Achieved a 95% success rate in project delivery timelines.  
  - Awarded "Rising Star" at the annual company gala for outstanding contributions.  

- **2022:**  
  - Exceeded goals by optimizing existing backend code, improving system performance by 25%.  
  - Conducted training sessions for junior developers, fostering knowledge sharing.  

- **2023:**  
  - Led a major overhaul of the API internal architecture, enhancing security proto

In [33]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever=vectorstore.as_retriever(search_kwargs={"k":25})
conversation_chain=ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory,
                                                        callbacks=[StdOutCallbackHandler()])

query="Who received the prestigious IIOTY award in 2023?"
result=conversation_chain.invoke({"question": query})
answer=result['answer']
print('\nAnswer:', answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Annual Performance History
- **2020:**  
  - Completed onboarding successfully.  
  - Met expectations in delivering project milestones.  
  - Received positive feedback from the team leads.

- **2021:**  
  - Achieved a 95% success rate in project delivery timelines.  
  - Awarded "Rising Star" at the annual company gala for outstanding contributions.  

- **2022:**  
  - Exceeded goals by optimizing existing backend code, improving system performance by 25%.  
  - Conducted training sessions for junior developers, fostering knowledge sharing.  

- **2023:**  
  - Led a major overhaul of the API internal architecture, enhancing security proto